In [0]:
with
f4108 as (
  SELECT 
  trim(IOLOTN) as BATCH_ID,
  CAST((IOUB04 / 10000) AS decimal(12, 1)) AS CompletedQuantity,
  IOLDSC
  FROM gms_us_lake.gmsgq_jde_proddta_F4108_ADT
)
select * from f4108
where batch_id like 'F4A003'

In [0]:
-- all quantity (UNRSTR_USAGE_QTY) of the batch_id needs to be in the high rack warehouse (HRL01), zero at other locations
with
inventory as (
  select
  inv.BATCH_ID,
  inv.UNRSTR_USAGE_QTY,
  inv.STORAGE_LOCATION_CD,
  inv.CREATED_ON_DT,
  split(inv.BATCH_ID, '/')[0] as BATCH_ID_PREFIX
  from gms_us_hub.txn_inventory_erp_glbl inv
  where STORAGE_LOCATION_CD like 'SL%'
  and trim(plant_id) = 'VN8'
  and inv.ACTIVE_FLAG = 'Y'
  and inv.CREATED_ON_DT > '2021-01-01'
  and inv.UNRSTR_USAGE_QTY != 0
),

f4108 as (
  SELECT 
  trim(IOLOTN) as BATCH_ID
  ,CAST((IOUB04 / 10000) AS decimal(38, 6)) AS CompletedQuantity
  ,IOLDSC as Description
  FROM gms_us_lake.gmsgq_jde_proddta_F4108_ADT
)

select * from inventory
left join f4108
on inventory.BATCH_ID_PREFIX = f4108.BATCH_ID